In [2]:
import pandas as pd
import nltk
import re

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


In [5]:
stemmer = PorterStemmer()
lemmatize = WordNetLemmatizer()


In [ ]:
paragraph = """
Chief Bola Ahmed Adekunle Tinubu GCFR (born 29 March 1952) is a Nigerian politician who is the 16th and current president of Nigeria.[1] He was the governor of Lagos State from 1999 to 2007; and senator for Lagos West in the Third Republic.

Tinubu spent his early life in southwestern Nigeria and later moved to the United States where he studied accounting at Chicago State University. He returned to Nigeria in the early 1990s and was employed by Mobil Nigeria as an accountant, before entering politics as a Lagos West senatorial candidate in 1992 under the banner of the Social Democratic Party. After dictator Sani Abacha dissolved the Senate in 1993, Tinubu became an activist campaigning for the return of democracy as a part of the National Democratic Coalition movement.

In the first post-transition Lagos State gubernatorial election, Tinubu won by a wide margin as a member of the Alliance for Democracy. Four years later, he won re-election to a second term. After leaving office in 2007, he played a key role in the formation of the All Progressives Congress in 2013. In 2023, he was elected president of Nigeria.
"""

In [7]:
# tokenization
nltk.download('stopwords')
nltk.download('punkt')

# sentences = nltk.sent_tokenize(paragraph)

[nltk_data] Downloading package stopwords to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
corpus = []
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review=review.lower()
    corpus.append(review)

In [ ]:
# to apply stemming
for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(stemmer.stem(word))

In [ ]:
stopwords.words('english')
nltk.download('wordnet')


In [ ]:
# paragraph = []

# for i in range(len(sentences)):
#     review = re.sub('[^a-zA-Z]', ' ', sentences[i])
#     review = review.lower()
#     review = review.split()
#     review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
#     review = ' '.join(review) 
#     paragraph.append(review)

In [ ]:
# to apply lemmatizer

for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(lemmatize.lemmatize(word))

In [ ]:
# bag of words

cv = CountVectorizer(binary=True, ngram_range=(3,3))

In [ ]:
X = cv.fit_transform(corpus)
cv.vocabulary_
sorted(cv.vocabulary_.items())


In [ ]:
X[0].toarray()

In [ ]:
corpus

In [ ]:
## TFIDF
# from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(ngram_range=(1,1), max_features=10)
X = tv.fit_transform(corpus)

In [ ]:
X[0].toarray()

In [3]:
# READ CSV
spam = pd.read_csv('spam_ham.csv', sep='\t', names=['labels', 'message'])

In [ ]:
# APPLY STEMMER
corpus_stem = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus_stem.append(review)

In [ ]:
# bag of words
cv = CountVectorizer(max_features=2500,binary=True)
X = cv.fit_transform(corpus_stem).toarray()

In [ ]:
y = pd.get_dummies(spam['labels'])
y = y.iloc[:, 1]

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
mnb = MultinomialNB().fit(X_train, y_train)
# prediction
y_pred = mnb.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
# classification_report(y_pred, y_test)
confusion_matrix(y_pred, y_test)

In [ ]:
# TFIDF

tv = TfidfVectorizer(max_features=2500, ngram_range=(1,2))
X = tv.fit_transform(corpus_stem).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
mnb = MultinomialNB().fit(X_train, y_train)
y_pred = mnb.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_pred, y_test)


### Word2vec

In [ ]:
%pip install gensim

In [21]:
import numpy as np

In [ ]:
# import gensim.downloader as api

# wv = api.load('word2vec-google-news-300')

In [6]:
corpus = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus.append(review)

In [7]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [8]:
words = []
for i in corpus:
    sent_token = sent_tokenize(i)
    for i in sent_token:
        words.append(simple_preprocess(i))

In [9]:
len(words)

5565

In [12]:
model = gensim.models.Word2Vec(words, window=5,min_count=2)

In [ ]:
model.wv.index_to_key

In [15]:
model.corpus_count

5565

In [16]:
model.epochs

5

In [18]:
model.wv.similar_by_word('feel')

('dont', 0.9996416568756104)

In [25]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    # sent = [word for word in doc if word in model.mv.index_to_key]
    # print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)
